In [2]:
## Imports

# utility modules
import glob
import os
import sys
import re


# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt

# specialty modules
import h5py
import pyproj

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Land ice applications: Getting familiar with ICESat-2 products over land ice

## 1. Introduction
In this tutorial, we're going to take a first look at some of the ICESat-2 data over land ice.  We're going to demonstrate how ALT06 segments correspond to ATL03 photons, and demonstrate the repeat-track structure of ATL06 data. '

## 1.1 Learning goals
Our learning goals include understanding:
* how ice-sheet surfaces are represented by ATL03 photons
* how ATL06 elevations correpond to ATL03 photon clouds
* how small-scale features appear in ATL06
* the repeat structure of ATL06
* how cross-track slope can affect elevation differences
and we'll also give:
* a sneak peak at the ATL11 product.

## 1.2 Tools presented and developed:
Along the way, we'll present:
* A reader for ATL03
* A reader for ATL06
* Geographic projections with pyproj
* Filtering ATL06 elevations using slope and quality parameters
* Mapping surface elevations and profiles with matplotlib
* Visualizing ATL03 and ATL06 together.

### 1.2.1 Modules Used:
If you look at the first cell in the notebook, you can see that we're importing:
* h5py: a library to read (and write) hdf5 data
* pyproj: a library of geographic corrections

We'll also us my pointCollection package, which contains code to read a variety of datatypes, which we'll use to read image data and to take a first look at the ATL11 product.

In [12]:
#! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
#sys.path.append(os.path.join('Land_Ice_Applications/readers'))
#import pointCollection as pc

In [3]:
cd

/home/jovyan


We'll also use Tyler Sutterley's excellent read_HDF5_ATL03 package (stored in the 'readers' directory, one level above the current directory.

In [4]:
cd Land_Ice_Applications/readers

/home/jovyan/Land_Ice_Applications/readers


In [5]:
from read_HDF5_ATL03 import read_HDF5_ATL03
from get_ATL03_x_atc import get_ATL03_x_atc

### 1.2.2 Data used
Data for this tutorial are stored in a shared drive, accessible to all tutorial participants.  If you're getting data for yourself, you'll need to put it in a consistent place, and change this cell to match your directory.

In [6]:
data_root='shared/FirnAndMelt/ATL03/Antarctica/'

In [7]:
import rasterio
import xarray as xr
import cartopy.crs as ccrs

In [8]:
cd

/home/jovyan


In [9]:
landsat8 = xr.open_rasterio('lores_20200120_127111_tif.tif')

In [10]:
plt.figure()
ax = plt.axes(projection=ccrs.SouthPolarStereo())
landsat8.plot(ax  = ax, transform = ccrs.SouthPolarStereo(), cmap='Greys_r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Image background

We'll also use an image mosaic as background for Antarcitca.  It's available in the shared space on jovyan in the MOA subdirectory

If you're running the script outside the hackweek.io world, you'll need to download the files from NSIDC.  I've posted a 1-km downscale of the raw (125-meter) mosaic, but the 750-meter mosaic here will do nicely:

https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0593_moa2009/geotiff/moa750_2009_hp1_v01.1.tif.gz

[Note that if you've gotten your own version of the MOA from NSIDC, you'll need to change the path to the file later in the tutorial to match where you've stored it]

#### ATL06 data
A set of data for the Pine Island Glacier are stored in the /PIG_ATL03 folder, under data-root folder on the Jupyter hub.
They cover a bounding box:
102 W to 98 W, 76 S to -75.4 S.  

#### ATL03 data
Two granules of ATL03 data from the same region are stored in the /PIG_ATL03 folder, under data-root folder on the Jupyter hub.

## 1.3 Geographic setting

Let's read the the Mosaic of Antarctica for the region around Pine Island Glacier.  The mosaic is in polar-stereographic coordinates, so we'll need to project the geographic coordinates of the box into that projection to decide what part of the mosaic to read.

In [16]:
spatial_extent = np.array([-102, -76, -98, -74.5])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
MOA=pc.grid.data().from_geotif(os.path.join(data_root, 'MOA','moa_2009_1km.tif'), bounds=[XR, YR])

# show the mosaic:
plt.figure()
MOA.show(cmap='gray', clim=[14000, 17000])
plt.plot(xy[0,:], xy[1,:])
plt.title('Mosaic of Antarctica for Pine Island Glacier')

[-76.  -74.5 -74.5 -76.  -76. ]
[ -98.  -98. -102. -102.  -98.]


NameError: name 'pc' is not defined

This is the Pine Island Glacier.  It flows from the top of the map to the bottom (actually east to west, but it's rotated in the Polar Stereographic projection used here).  The striped area running from the middle of the page to the bottom is the Pine Island Ice Shelf, where the ice is floating.  We expect to see crevasses at the edges of the ice shelf (or really, anywhere in the ice shelf).  This area is often cloudy, so we expect to have trouble seeing the surface a lot of the time.


# 2. Data in along-track format

# 2.1 ATL03 elevations

Before we start looking at the ATL06 data we've downloaded, let's have a look at some of the ATL03 data that were used to make them.  One of the source ATL03 files is in the shared folder, and we'll read it with Tyler Sutterley's excellent "read_HDF5_ATL03" function.

In [20]:
cd 

/home/jovyan


In [21]:
cd 'shared/FirnAndMelt/ATL03/Antarctica/'

/srv/shared/FirnAndMelt/ATL03/Antarctica


In [24]:
# read the data:
#rgt="0027"
#cycle="06"
# read the IS2 data with Tyler's ATL03 reader:
ATL03_file=  'processed_ATL03_20200203123745_05920612_003_01.h5'
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)

The data are returned in a set of dictionaries that mimic the structure of an ATL03 file.  To help visualize the data, we're going to calculate an along-track coordinate for every photon in the cloud (x_atc).  This is a slightly complex job, and there's a helper function in the readers directory that you can look at if you want the details.

In [25]:
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

### 2.1.1 Plotting ATL03 photons
Now let's plot the ATL03 photons.  We'll plot all the photon heights as small black dots, then plot the photons that the ATL03 land-ice signal finder designates as surface (with low, medium, or high confidence) in green.  

In [26]:
#-- select the 2l beam from ATL03
D3 = IS2_atl03_mds['gt2r']

#-- create scatter plot of photon data (e.g., photon elevation vs x_atc)
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(D3['heights']['lat_ph'], D3['heights']['h_ph'],'k.',markersize=0.25, label='all photons')
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
ax.plot(D3['heights']['lat_ph'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
h_leg=ax.legend()

ax.set_xlabel('x_atc, m')
ax.set_ylabel('h, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

What we see here is a fat black bar, representing all the photons that were close to the surface (in a ~200 m window), with a green line representing the photons flagged by ATL03 as representing the surface. If we zoom in (using the square button on the right) we can see that the black bar is made up of lots of individual photons, with a darker line that represents the surface.  

In [31]:
plt.figure()
ax = plt.axes(projection=ccrs.SouthPolarStereo())
landsat8.plot(ax  = ax, transform = ccrs.SouthPolarStereo(), cmap='Greys_r')
ax.set_extent([67.5, 68.5, -72.3, -72], ccrs.PlateCarree())
plt.scatter(D3['heights']['lon_ph'], D3['heights']['lat_ph'], transform=ccrs.PlateCarree())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1.2 Cloudy tracks and singnal-finding blunders

In [38]:

TEP=(D3['heights']['signal_conf_ph'][:,3] <-1)   
BG=(D3['heights']['signal_conf_ph'][:,3] ==0)   |  (D3['heights']['signal_conf_ph'][:,3] ==1)
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2


array([121.13486, 121.03395, 121.3556 , ..., 102.63397, 104.63813,
       102.27638], dtype=float32)

In [43]:
%matplotlib widget
plt.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
import pandas as pd

In [54]:
selection = pd.Series(data = D3['heights']['h_ph'][LMH], index = D3['heights']['x_atc'][LMH])

In [64]:
selection_var  = selection.rolling(100).var()

In [65]:
%matplotlib widget
selection_var.plot()
selection_var.where(selection_var.values>1).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
lake_idx  = np.argwhere(selection_var.values>1)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [67]:
%matplotlib widget
plt.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
plt.plot(D3['heights']['x_atc'][LMH][lake_idx], D3['heights']['h_ph'][LMH][lake_idx],'b.',markersize=0.5, label='flagged photons')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …